<a href="https://colab.research.google.com/github/MaggieAppleton/Colab-Notebooks/blob/main/Tool_Loops_and_Agents_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Agent Loop Patterns - Practice Exercises


In [ ]:
# Setup

%pip install anthropic wikipedia

import anthropic
from google.colab import userdata
from IPython.display import display, Markdown
import json
import re
import random
import time
import wikipedia

api_key = userdata.get('ANTHROPIC_API_KEY')
claude_client = anthropic.Anthropic(api_key=api_key)

model="claude-3-5-haiku-latest"
max_tokens=1000

print("👍 Claude setup")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=eb90accf5e1c97a7c6516a64e84e6ac474d094e19d40a236a10f2a5b7cb722c0
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
👍 Claude setup


# ✅ EXERCISE 1: Basic Loop Pattern - Counting to 10

In [ ]:
# GOAL: Master the fundamental conversation loop pattern

# CHALLENGE:
# Write a function that asks Claude to count from 1 to 10, but Claude can only
# say one number per response. You need to maintain the conversation until
# Claude reaches 10 and says "DONE".

# ACCEPTANCE CRITERIA:
# ✅ Function maintains conversation state across multiple responses
# ✅ Claude counts 1, 2, 3... 10 in separate responses
# ✅ Loop terminates when Claude says "DONE"
# ✅ Function returns the complete counting sequence
# ✅ Handles edge cases (what if Claude skips a number?)

# LEARNING OBJECTIVES:
# - Basic message state management
# - Loop termination conditions
# - Conversation continuity

def counting_loop():
    """
    TODO: Implement the counting loop

    System prompt should ask Claude to count to 10, one number per response,
    and say "DONE" when finished.

    Return: List of all Claude's responses
    """

    system_prompt = """
    We're going to count from 1 to 10, one number at a time. Only return a single number in your response. It should be the next number in the sequence. When you've finished counting, return DONE.
    """

    messages = [{ "role": "user", "content": "Begin counting" }]
    responses = []

    while True:

      # Send Claude the current messages
      response = claude_client.messages.create(
          messages=messages,
          system=system_prompt,
          max_tokens=max_tokens,
          model=model,
      )

      # If there is a response, append the response to 'messages' and to 'responses'
      if response.content[0].text:
          messages.append({ "role": "assistant", "content": response.content[0].text })
          responses.append(response.content[0].text)

          print(response.content[0].text)

      if "DONE" in response.content[0].text:
          return responses
          break

      else:
          messages.append({ "role": "user", "content": "Next number" })

# Test your solution
responses = counting_loop()
print("Claude's counting responses:", responses)

1
2
3
4
5
6
7
8
9
10
DONE
Claude's counting responses: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'DONE']


# ✅ EXERCISE 2: Multi-Step Reasoning with Termination Tags

In [ ]:
# GOAL: Learn to use structured tags for controlling agent behavior

# CHALLENGE:
# Create a function that gives Claude a math word problem. Claude must solve it
# step-by-step, showing one step per response, then provide a final answer in
# <final_answer> tags.

# EXAMPLE PROBLEM:
# "Sarah has 24 apples. She gives 1/3 to her friend, then buys 18 more apples.
# How many apples does she have now?"

# ACCEPTANCE CRITERIA:
# ✅ Claude shows work step-by-step (multiple responses)
# ✅ Each step builds on the previous ones
# ✅ Final answer is wrapped in <final_answer> tags
# ✅ Function extracts and returns just the final numerical answer
# ✅ Handles parsing errors gracefully

# LEARNING OBJECTIVES:
# - Structured output with XML tags
# - Multi-step reasoning chains
# - Text parsing and extraction

def extract_answer(text):
  """Extract content between <final_answer> tags"""

  pattern = r'<final_answer>(.*?)</final_answer>'
  match = re.search(pattern, text, re.DOTALL)

  if match:
    return match.group(1).strip()
  return None

def extract_reasoning(text):
  pattern = r'<reasoning>(.*?)</reasoning>'
  match = re.search(pattern, text, re.DOTALL)

  if match:
    return match.group(1).strip()
  return None

def math_solver(word_problem):
    """
    TODO: Implement the step-by-step math solver

    Args:
        word_problem (str): A math word problem to solve

    Return: The final numerical answer (as string or number)
    """

    system_prompt = """
    You are a mathematics assistant. Help the user solve their math problem by breaking it down and reasoning through it step-by-step.

    Guidelines:
    - Show your work clearly at each step
    - Use the available tools when you need to perform calculations
    - Wrap your reasoning in <reasoning> tags
    - When you have the final answer, wrap it in <final_answer> tags
    - Be precise and show intermediate results
    """
    messages=[{ "role": "user", "content": word_problem}]

    while True:

      response = claude_client.messages.create(
          model=model,
          system=system_prompt,
          max_tokens=max_tokens,
          messages=messages
      )

      # If we get a response from Claude, append it to the messages list.
      if (response.content and len(response.content) > 0 and hasattr(response.content[0], 'text')) and response.content[0].text:

          text_content = response.content[0].text
          messages.append({"role": "assistant", "content": text_content})
          # print(f"Claude says: {text_content}")

          # If reasoning step, print reasoning
          if "<reasoning>" in text_content:
            reasoning = extract_reasoning(text_content)
            print(f"\nReasoning: {reasoning}")

          # If the response contains a final answer, break and return the answer
          if "<final_answer>" in text_content:
            answer = extract_answer(text_content)
            return answer
          else:
            messages.append({"role": "user", "content": "Continue to the next step"})

# Test problems
test_problems = [
    "Sarah has 24 apples. She gives 1/3 to her friend, then buys 18 more apples. How many apples does she have now?",
    "A store sells shirts for $15 each. If they have a 20% discount and someone buys 3 shirts, how much do they pay?",
    "Tom runs 3 miles every day for a week, then 5 miles every day the next week. How many total miles did he run?"
]

for problem in test_problems:
    answer = math_solver(problem)
    print(f"Problem: {problem}")
    print(f"Answer: {answer}\n")


Reasoning: 1. First, Sarah starts with 24 apples

2. She gives 1/3 of her apples to her friend
   • 1/3 of 24 = 24 ÷ 3 = 8 apples given away
   • Remaining apples = 24 - 8 = 16 apples

3. Then she buys 18 more apples
   • 16 + 18 = 34 apples

4. Let's verify the steps:
   • Initial apples: 24
   • Apples given away: 8
   • Apples remaining: 16
   • Apples bought: 18
   • Final number of apples: 16 + 18 = 34
Problem: Sarah has 24 apples. She gives 1/3 to her friend, then buys 18 more apples. How many apples does she have now?
Answer: 34 apples


Reasoning: 1. Original price of one shirt: $15
2. Calculate the discount amount:
   • 20% of $15 = $15 × 0.20 = $3 discount per shirt
3. Discounted price of one shirt:
   • $15 - $3 = $12 per shirt
4. Calculate total for 3 shirts:
   • $12 × 3 = $36
Problem: A store sells shirts for $15 each. If they have a 20% discount and someone buys 3 shirts, how much do they pay?
Answer: $36


Reasoning: 1. First week: Tom runs 3 miles per day
• Days in a 

# EXERCISE 3: Tool Use Loop - Calculator Agent

In [ ]:
# GOAL: Master the tool use loop pattern with a simple calculator

# CHALLENGE:
# Build a calculator tool and an agent that can solve complex expressions by
# breaking them into multiple calculator operations.

# EXAMPLE: "((15 + 27) * 3) - (18 / 2)" should become multiple calculator calls

# ACCEPTANCE CRITERIA:
# ✅ Calculator tool handles +, -, *, / operations
# ✅ Agent breaks complex expressions into simple operations
# ✅ Tool use loop continues until expression is fully solved
# ✅ Final answer is clearly identified
# ✅ Handles tool errors gracefully

# LEARNING OBJECTIVES:
# - Tool definition and execution
# - Tool use detection and response handling
# - Complex problem decomposition

def calculator_tool(operation, num1, num2):
    """
    Simple calculator tool - Claude will call this
    """
    try:
        if operation == "add":
            return num1 + num2
        elif operation == "subtract":
            return num1 - num2
        elif operation == "multiply":
            return num1 * num2
        elif operation == "divide":
            if num2 == 0:
                return "Error: Division by zero"
            return num1 / num2
        else:
            return "Error: Unknown operation"
    except Exception as e:
        return f"Error: {str(e)}"

# Define the tool schema for Claude
calculator_tool_schema = {
    # Fill in
}

def calculator_agent(expression):
    """
    TODO: Implement the calculator agent

    Args:
        expression (str): Complex mathematical expression to solve

    Return: Final numerical result
    """

    # Your code here
    pass

# Test expressions
test_expressions = [
    "((15 + 27) * 3) - (18 / 2)",
    "100 / 4 + 50 * 2",
    "(45 - 15) * (20 / 4) + 10"
]

# for expr in test_expressions:
#     result = calculator_agent(expr)
#     print(f"Expression: {expr}")
#     print(f"Result: {result}\n")


# EXERCISE 4: Research Agent - Person Profile Builder

In [ ]:
# GOAL: Build a multi-step research agent that gathers related information

# CHALLENGE:
# Create an agent that researches a famous person by:
# 1. Getting basic info about the person
# 2. Researching their birthplace
# 3. Researching their main profession/field
# 4. Compiling a comprehensive profile

# ACCEPTANCE CRITERIA:
# ✅ Agent makes multiple research calls building on previous info
# ✅ Each search is relevant to previous findings
# ✅ Final profile includes person, birthplace, and profession details
# ✅ Handles cases where information isn't found
# ✅ Profile is well-structured and comprehensive

# LEARNING OBJECTIVES:
# - Multi-step information gathering
# - Context building across tool calls
# - Information synthesis
# - Real API integration and error handling

In [ ]:
# Helper function to test the Wikipedia tool directly
def test_wikipedia_tool():
    """Test the Wikipedia research tool with various queries"""
    test_queries = [
        "Albert Einstein",
        "Ulm, Germany",
        "Theoretical Physics",
        "Nonexistent Person XYZ"  # Test error handling
    ]

    print("🧪 Testing Wikipedia Research Tool")
    print("=" * 50)

    for query in test_queries:
        print(f"\nSearching for: {query}")
        result = wikipedia_research_tool(query)
        print(result[:200] + "..." if len(result) > 200 else result)
        print("-" * 30)

# Uncomment to test the Wikipedia tool:
# test_wikipedia_tool()

In [ ]:
def wikipedia_research_tool(search_query, sentences=3):
    """
    Real Wikipedia research tool using the wikipedia package

    Args:
        search_query (str): What to search for on Wikipedia
        sentences (int): Number of sentences to return from summary

    Returns:
        str: Wikipedia summary or error message
    """
    try:
        # Handle disambiguation and search issues
        try:
            # First try exact search
            page = wikipedia.page(search_query)
            summary = wikipedia.summary(search_query, sentences=sentences)
            return f"Title: {page.title}\nSummary: {summary}\nURL: {page.url}"

        except wikipedia.DisambiguationError as e:
            # If disambiguation, try the first option
            first_option = e.options[0]
            page = wikipedia.page(first_option)
            summary = wikipedia.summary(first_option, sentences=sentences)
            return f"Title: {page.title}\nSummary: {summary}\nURL: {page.url}\nNote: Disambiguated to '{first_option}'"

        except wikipedia.PageError:
            # If page not found, try search
            search_results = wikipedia.search(search_query, results=3)
            if search_results:
                # Try the first search result
                page = wikipedia.page(search_results[0])
                summary = wikipedia.summary(search_results[0], sentences=sentences)
                return f"Title: {page.title}\nSummary: {summary}\nURL: {page.url}\nNote: Found via search for '{search_query}'"
            else:
                return f"No Wikipedia articles found for: {search_query}"

    except Exception as e:
        return f"Wikipedia search error for '{search_query}': {str(e)}"

research_tool_schema = {
    "name": "wikipedia_research",
    "description": "Search Wikipedia for information about people, places, or topics",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_query": {
                "type": "string",
                "description": "What to search for on Wikipedia"
            },
            "sentences": {
                "type": "integer",
                "description": "Number of sentences to return from summary (default: 3)",
                "default": 3
            }
        },
        "required": ["search_query"]
    }
}

def research_agent(person_name):
    """
    TODO: Implement the research agent using real Wikipedia data

    The agent should:
    1. Search for the person to get basic biographical info
    2. Extract birthplace from the results and research that location
    3. Extract profession/field and research that topic
    4. Compile all information into a comprehensive profile

    Args:
        person_name (str): Name of famous person to research

    Return: Comprehensive profile dictionary with keys:
            'person', 'birthplace', 'profession', 'summary'
    """

    # Your code here - implement the multi-step research process
    # Remember to:
    # - Use the wikipedia_research_tool function
    # - Build context from each search to inform the next
    # - Handle cases where information might not be found
    # - Extract key details (birthplace, profession) from Wikipedia summaries
    # - Compile everything into a structured profile

    pass

# Enhanced test with more diverse examples
test_people = [
    "Albert Einstein",
    "Marie Curie",
    "Leonardo da Vinci",
    "Frida Kahlo",
    "Nelson Mandela",
    "Jane Austen"
]

# Uncomment to test with real Wikipedia data:
# for person in test_people:
#     print(f"Researching {person}...")
#     profile = research_agent(person)
#     print(f"Profile for {person}:")
#     print(json.dumps(profile, indent=2))
#     print("-" * 50)

# EXERCISE 5: Validation Loop - Creative Writing with Quality Control

In [ ]:
# GOAL: Learn to handle rejection and iteration in agent loops

# CHALLENGE:
# Create a creative writing agent that generates short stories, but they must
# pass a "quality checker" tool that randomly accepts or rejects them.
# The agent should keep refining until the story is accepted.

# ACCEPTANCE CRITERIA:
# ✅ Agent generates creative stories on given topics
# ✅ Stories are checked by quality validator
# ✅ Agent refines rejected stories based on feedback
# ✅ Loop continues until story passes validation
# ✅ Maximum iteration limit prevents infinite loops
# ✅ Returns final accepted story

# LEARNING OBJECTIVES:
# - Handling tool rejection/failure
# - Iterative refinement
# - Feedback incorporation
# - Loop termination safeguards

In [ ]:
def quality_checker_tool(story):
    """
    Mock quality checker - randomly accepts or rejects stories
    In reality, this could be a more sophisticated validator
    """

    # Random acceptance with some basic checks
    if len(story.strip()) < 50:
        return {
            "approved": False,
            "feedback": "Story too short. Please write at least 50 characters."
        }

    if "the end" not in story.lower():
        return {
            "approved": False,
            "feedback": "Story needs a clear ending. Please add 'The End' or similar conclusion."
        }

    # Random approval for stories that meet basic criteria
    approved = random.random() > 0.4  # 60% approval rate

    if approved:
        return {
            "approved": True,
            "feedback": "Great story! Well structured and engaging."
        }
    else:
        feedback_options = [
            "Needs more character development and emotional depth.",
            "The plot could be more engaging with additional conflict.",
            "Consider adding more descriptive details to set the scene.",
            "The dialogue could be more natural and realistic."
        ]
        return {
            "approved": False,
            "feedback": random.choice(feedback_options)
        }

quality_checker_schema = {
    "name": "quality_checker",
    "description": "Check if a creative story meets quality standards",
    "input_schema": {
        "type": "object",
        "properties": {
            "story": {
                "type": "string",
                "description": "The story to check"
            }
        },
        "required": ["story"]
    }
}

def creative_writing_agent(topic, max_iterations=5):
    """
    TODO: Implement the creative writing agent with quality control

    Args:
        topic (str): Topic for the story
        max_iterations (int): Maximum refinement attempts

    Return: Dictionary with 'final_story', 'iterations_used', 'approved'
    """

    # Your code here
    pass

# Test topics
test_topics = [
    "A robot learning to paint",
    "Time travel gone wrong",
    "The last bookstore on Earth"
]

# for topic in test_topics:
#     result = creative_writing_agent(topic)
#     print(f"Topic: {topic}")
#     print(f"Iterations: {result['iterations_used']}")
#     print(f"Approved: {result['approved']}")
#     print(f"Story: {result['final_story'][:200]}...")
#     print()

# EXERCISE 6: Multi-Tool Agent - Trip Planning Assistant

In [ ]:
# GOAL: Master complex workflows with multiple tools and decision making

# CHALLENGE:
# Build a trip planning agent with three tools:
# - flight_search: Find flights between cities
# - cost_calculator: Calculate total trip costs
# - itinerary_writer: Save trip plans to "file"

# The agent should plan a complete trip with flights, cost estimates, and itinerary.

# ACCEPTANCE CRITERIA:
# ✅ Agent searches for flights for given destination
# ✅ Calculates total costs including flights and estimated expenses
# ✅ Creates and saves a detailed itinerary
# ✅ Uses all three tools appropriately
# ✅ Handles cases where flights aren't available
# ✅ Returns comprehensive trip plan

# LEARNING OBJECTIVES:
# - Multi-tool coordination
# - Complex workflow management
# - Decision making between tools
# - Error handling across multiple systems

In [ ]:
def flight_search_tool(origin, destination, dates=None):
    """Mock flight search tool"""

    # Mock flight data
    mock_flights = {
        ("new york", "paris"): {"price": 650, "duration": "8h 30m", "airline": "Air France"},
        ("london", "tokyo"): {"price": 720, "duration": "11h 45m", "airline": "British Airways"},
        ("los angeles", "sydney"): {"price": 890, "duration": "15h 20m", "airline": "Qantas"},
        ("chicago", "rome"): {"price": 580, "duration": "9h 15m", "airline": "Lufthansa"},
        ("san francisco", "london"): {"price": 620, "duration": "10h 30m", "airline": "Virgin Atlantic"}
    }

    key = (origin.lower(), destination.lower())
    reverse_key = (destination.lower(), origin.lower())

    if key in mock_flights:
        return mock_flights[key]
    elif reverse_key in mock_flights:
        return mock_flights[reverse_key]
    else:
        return {"error": f"No flights found between {origin} and {destination}"}

def cost_calculator_tool(flight_cost, days, daily_budget=100):
    """Calculate total trip cost"""

    try:
        accommodation = days * 80  # $80/night average
        food = days * daily_budget * 0.4  # 40% of daily budget for food
        activities = days * daily_budget * 0.6  # 60% for activities

        total = flight_cost + accommodation + food + activities

        return {
            "flight_cost": flight_cost,
            "accommodation": accommodation,
            "food": food,
            "activities": activities,
            "total_cost": total,
            "cost_breakdown": f"Flight: ${flight_cost}, Accommodation: ${accommodation}, Food: ${food}, Activities: ${activities}"
        }
    except Exception as e:
        return {"error": f"Cost calculation failed: {str(e)}"}

def itinerary_writer_tool(destination, days, flight_info, cost_info):
    """Create and 'save' trip itinerary"""

    try:
        itinerary = f"""
TRIP ITINERARY - {destination.upper()}

FLIGHT DETAILS:
- Airline: {flight_info.get('airline', 'TBD')}
- Duration: {flight_info.get('duration', 'TBD')}
- Cost: ${flight_info.get('price', 'TBD')}

TRIP DURATION: {days} days

COST BREAKDOWN:
{cost_info.get('cost_breakdown', 'Cost calculation pending')}
Total Budget: ${cost_info.get('total_cost', 'TBD')}

DAILY PLAN:
""" + "\n".join([f"Day {i+1}: Explore {destination} - Activities and sightseeing" for i in range(days)])

        itinerary += f"\n\nItinerary saved successfully for {destination} trip!"

        return {
            "success": True,
            "itinerary": itinerary,
            "filename": f"{destination.lower()}_trip_itinerary.txt"
        }
    except Exception as e:
        return {"error": f"Failed to create itinerary: {str(e)}"}

# Tool schemas
trip_tools_schemas = [
    {
        "name": "flight_search",
        "description": "Search for flights between two cities",
        "input_schema": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "Departure city"},
                "destination": {"type": "string", "description": "Arrival city"},
                "dates": {"type": "string", "description": "Travel dates (optional)"}
            },
            "required": ["origin", "destination"]
        }
    },
    {
        "name": "cost_calculator",
        "description": "Calculate total trip costs including flights and daily expenses",
        "input_schema": {
            "type": "object",
            "properties": {
                "flight_cost": {"type": "number", "description": "Cost of flights"},
                "days": {"type": "number", "description": "Number of days for the trip"},
                "daily_budget": {"type": "number", "description": "Daily budget for food and activities"}
            },
            "required": ["flight_cost", "days"]
        }
    },
    {
        "name": "itinerary_writer",
        "description": "Create and save a detailed trip itinerary",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination": {"type": "string", "description": "Trip destination"},
                "days": {"type": "number", "description": "Number of days"},
                "flight_info": {"type": "object", "description": "Flight information"},
                "cost_info": {"type": "object", "description": "Cost breakdown information"}
            },
            "required": ["destination", "days", "flight_info", "cost_info"]
        }
    }
]

def trip_planner(origin, destination, days, daily_budget=150):
    """
    TODO: Implement the multi-tool trip planning agent

    Args:
        origin (str): Departure city
        destination (str): Destination city
        days (int): Number of days for trip
        daily_budget (int): Daily budget for food and activities

    Return: Complete trip plan dictionary
    """

    # Your code here
    pass

# Test trip plans
test_trips = [
    ("New York", "Paris", 7, 200),
    ("London", "Tokyo", 10, 150),
    ("Los Angeles", "Sydney", 14, 180)
]

# for origin, dest, days, budget in test_trips:
#     plan = trip_planner(origin, dest, days, budget)
#     print(f"\nTrip Plan: {origin} → {dest}")
#     print("="*50)
#     if plan:
#         print(json.dumps(plan, indent=2))
#     print()